In [ ]:
import time
import asyncio
import tracemalloc
from loguru import logger
from tqdm import tqdm, trange
from random import randint, choice
from proxy_client import ProxyProvider
from requests_html import AsyncHTMLSession

tracemalloc.start()

def setup_logger(file_name):
    logger.remove()
    logger.add(f"./logs/{file_name}-debug.log", format="{time} {level} {message}", level="DEBUG", enqueue=True)
    logger.add(f"./logs/{file_name}-info.log", format="{time} {level} {message}", level="INFO", enqueue=True, backtrace=True)
    logger.add(f"./logs/{file_name}-error.log", format="{time} {level} {message}", level="ERROR", enqueue=True, backtrace=True, diagnose=True)

setup_logger("proxy-refine")


proxy_provider = ProxyProvider()
asession = AsyncHTMLSession()
urls = [
    "https://divar.ir/s/tehran/jobs",
    "https://instagram.com", 
    "https://www.googleapis.com/discovery/v1/apis/drive/v3/rest"
    "https://www.tse.ir",
    "http://tsetmc.ir",
    "http://tsetmc.ir/Loader.aspx?ParTree=15",
    "http://www.fipiran.com",
]
# urls = ["https://www.googleapis.com/discovery/v1/apis/drive/v3/rest", "https://balad.ir", 
#         "https://google.com", "https://youtube.com", "https://facebook.com", "https://baidu.com",
#         "https://wikipedia.org", "https://twitter.com", "https://yahoo.com", "https://instagram.com",
#         "https://amazon.com"]
async def getit(proxy_provider):
    i = 0
    while i < 100000:
        logger.debug(f"starting {i}")
        i += 1
        try:
            url = choice(urls)
            protocol = url.split("://")[0]
            if i % 5 == 0:
                p = proxy_provider.get_proxy(protocol)
            else:
                p = proxy_provider.get_bad_proxy(protocol)
            if p == None or p == "None":
                time.sleep(10)
                continue
            r = await asession.get(f"{protocol}://google.com", proxies={protocol: p}, timeout=15)
            if "google.com" in r.text:
                r = await asession.get(url, proxies={protocol: p}, timeout=15)
                print(protocol, p)
                if r.status_code == 403:
                    proxy_provider.bad_ip(protocol, p)
            else:
                proxy_provider.bad_ip(protocol, p)
            del p
            del r
        except:
            proxy_provider.bad_ip(protocol, p)
            del p

tasks = []

for i in trange(50):
    tasks.append(getit(proxy_provider))
results = await asyncio.wait(tasks)

100%|██████████| 50/50 [00:00<00:00, 33808.67it/s]


http 209.45.61.108:3128
https 163.172.136.226:8811
http 36.67.218.105:53281
https 118.172.181.147:34388
http 134.122.103.6:1234
https 144.217.101.242:3129
http 212.220.216.70:8080
https 45.175.139.89:999
http 95.179.185.213:3128
http 199.188.93.202:8000
http 167.71.5.83:8080
http 191.232.233.45:3128
http 46.34.175.224:8080
http 104.244.77.254:8080
http 89.28.197.124:8080
http 196.223.162.226:80
http 177.73.51.7:8080
https 116.197.131.26:8080
http 51.158.99.51:8811
http 46.35.184.187:61003
http 159.203.44.177:3128
http 200.89.174.104:8080
https 199.195.248.24:8080
http 37.59.61.18:8080
http 37.192.194.50:50165
http 104.131.14.247:80
http 36.55.226.137:3128
http 193.160.214.29:3169
https 116.197.131.28:8080
http 51.158.68.133:8811
http 58.220.95.91:9443
https 51.158.68.26:8811
https 118.27.15.118:3128
https 101.109.255.97:44351
https 103.226.49.114:8080
https 113.23.138.155:8989
http 116.197.131.26:8080
http 88.198.50.103:8080
https 5.160.150.68:8080
http 36.55.226.146:3128
http 185.134.

In [ ]:
print(2)

In [ ]:
import requests
import re
from time import gmtime, strftime, sleep
from tqdm import trange, tqdm
import time
import asyncio
import tracemalloc
from loguru import logger
from tqdm import tqdm, trange
from random import randint, choice
from proxy_client import ProxyProvider
from requests_html import AsyncHTMLSession
tracemalloc.start()

urls = []
with open("proxy_url.txt", "r") as f:
    for line in f.readlines():
        urls.append(line[:-1])

def setup_logger(file_name):
    logger.remove()
    logger.add(f"./logs/{file_name}-debug.log", format="{time} {level} {message}", level="DEBUG", enqueue=True)
    logger.add(f"./logs/{file_name}-info.log", format="{time} {level} {message}", level="INFO", enqueue=True, backtrace=True)
    logger.add(f"./logs/{file_name}-error.log", format="{time} {level} {message}", level="ERROR", enqueue=True, backtrace=True, diagnose=True)

setup_logger("proxy")


asession = AsyncHTMLSession()
q_url = asyncio.Queue()
q_goods = asyncio.Queue()
q_proxies = asyncio.Queue()
for url in tqdm(urls):
    await q_url.put(url)


async def getit(q_url, q_goods, q_proxies):
    while not q_url.empty():
        url = await q_url.get()
        logger.debug(f"start {url}")
        try:
            resp = await asession.get(url, timeout=15)
            regs = re.findall(r'((?:\d{1,3}\.){3}\d{1,3}):(\d+)', resp.text)
            for reg in regs:
                await q_proxies.put(reg[0] + ":" + reg[1])
            await q_goods.put(url)
            logger.info(f"done url {url}")
        except:
            logger.debug(f"bad url {url}")
        q_url.task_done()
        

tasks = []
for i in trange(50):
    logger.info("start runner")
    tasks.append(getit(q_url, q_goods, q_proxies))
results = await asyncio.wait(tasks)

proxies = []
while not q_proxies.empty():
    p = await q_proxies.get()
    proxies.append(p)
    
goods = []
while not q_goods.empty():
    g = await q_goods.get()
    goods.append(g)

100%|██████████| 50/50 [00:00<00:00, 1794.98it/s]
